TODO:
- store and plot hashgnn embeddings using x, y coordinates
    - [Plot embedddings with graphistry](https://github.com/graphistry/pygraphistry/blob/master/demos/demos_databases_apis/umap_learn/umap_learn.ipynb)
- include more context into plots: 
    - Taxon HAS_PARENT edges
    - SOTU SEQUENCE_ALIGNMENT edges
    - Remove Palmprints and only keep SOTUs


In [1]:
# Notebook config
import sys
if '../' not in sys.path:
    sys.path.append("../")
%load_ext dotenv
%dotenv

# Imports
import graphistry
import os
from queries import feature_queries
import pandas as pd


In [2]:
graphistry.register(api=3, username=os.getenv('GRAPHISTRY_USERNAME'), password=os.getenv('GRAPHISTRY_PASSWORD'))

In [3]:
dataset_dir = '/mnt/graphdata/datasets/data-v1/palmprint-host-dataset_0.1/'
results_dir = '/mnt/graphdata/results/link_prediction/0.1/ft-eng/'

df_has_host = feature_queries.get_features_from_file(
    file_name='sotu_has_host_metadata_edges.csv',
    dir_name=dataset_dir
)
df_has_host = df_has_host.astype(str)

df_taxon = feature_queries.get_features_from_file(
    file_name='taxon_nodes.csv',
    dir_name=dataset_dir
)
df_taxon = df_taxon.astype(str)

df_taxon = df_has_host[['targetAppId']].merge(
    df_taxon,
    left_on='targetAppId',
    right_on='appId',
    how='left',
).dropna().compute()


df_palmprint = feature_queries.get_features_from_file(
    file_name='sotu_nodes.csv',
    dir_name=dataset_dir
)
df_palmprint = df_palmprint.astype(str)

df_palmprint = df_has_host[['sourceAppId']].merge(
    df_palmprint,
    left_on='sourceAppId',
    right_on='appId',
    how='left',
).dropna().compute()


df_has_parent = feature_queries.get_features_from_file(
    file_name='has_parent_edges.csv',
    dir_name=dataset_dir
)
df_has_parent = df_has_parent.astype(str)

# df_has_parent = df_has_parent[['sourceAppId']].merge(
#     df_taxon,
#     left_on='sourceAppId',
#     right_on='appId',
#     how='left',
# ).dropna()
# print(len(df_has_parent))


df_pred_has_host = feature_queries.get_features_from_file(
    file_name='approx_predictions.csv',
    dir_name=results_dir
)

# print('HAS_HOST_METADATA')
# print(len(df_has_host))
# print(df_has_host.head())

# print('HAS_PARENT')
# print(len(df_has_parent))
# print(df_has_parent.head())

# print('Palmprint')
# print(len(df_palmprint))
# print(df_palmprint.head())

# print('Taxon')
# print(len(df_taxon))
# print(df_taxon.head())

No local cache file found /mnt/graphdata/datasets/data-v1/palmprint-host-dataset_0.1/sotu_has_host_metadata_edges.csv


AttributeError: 'NoneType' object has no attribute 'astype'

In [5]:
# df_taxon_reudced = df_taxon[['appId', 'features']]
# df_palmprint_reduced = df_palmprint[['appId', 'features']]

df_taxon['type'] =  df_taxon['labels']
df_palmprint['type'] =  df_palmprint['labels']

merged_nodes = pd.concat([
    df_taxon[['appId', 'nodeId', 'type', 'rank']],
    df_palmprint[['appId', 'nodeId', 'type', 'centroid']],
]).drop_duplicates(subset=['appId'])
classes = set(col for col in merged_nodes['type'].values)
print(len(classes))
print(classes)



3
{"['Taxon', 'Host']", "['Palmprint', 'SOTU']", "['Palmprint']"}


In [8]:
# df_pred_has_host.astype(str)

# df_pred_has_host['sourceNodeId'] = df_pred_has_host['node1']
# df_pred_has_host['targetNodeId'] = df_pred_has_host['node2']
# df_pred_has_host['weight'] = df_pred_has_host['probability']

# df_pred_has_host['sourceNodeId'] = df_pred_has_host['sourceNodeId'].astype(str)
# df_pred_has_host['targetNodeId'] = df_pred_has_host['targetNodeId'].astype(str)
# df_pred_has_host['weight'] = df_pred_has_host['weight'].astype(str)
# df_pred_has_host['relationshipType'] = 'PREDICTED_HAS_HOST'

# print(df_pred_has_host.head())
# print(len(df_pred_has_host))

# df_pred_has_host = df_pred_has_host.loc[:, ~df_pred_has_host.columns.str.contains('node1')]
# df_pred_has_host = df_pred_has_host.loc[:, ~df_pred_has_host.columns.str.contains('node2')]
# df_pred_has_host = df_pred_has_host.loc[:, ~df_pred_has_host.columns.str.contains('probability')]

# df_pred_has_host[''] = df_pred_has_host.index.rename({
#     'node1': 'sourceNodeId',
#     'node2': 'targetNodeId',
#     'probability': 'weight',
# })

merged_edges = pd.concat([
    df_has_host.compute(),
    # df_has_parent.compute(),
    # df_pred_has_host.compute(),
]).drop_duplicates(subset=['sourceAppId', 'targetAppId'])



In [296]:
print(merged_edges.head())

  sourceNodeId targetNodeId relationshipType sourceAppId targetAppId  \
0      7682523     10228304         HAS_HOST_METADATA        u509     1807597   
1      7682571      8927404         HAS_HOST_METADATA      u13821      211044   
2      7694378      8794922         HAS_HOST_METADATA     u738728       60711   
3      7694393     11002401         HAS_HOST_METADATA     u102609     2697049   
4      7695465      8761715         HAS_HOST_METADATA     u466651        7227   

                   weight weightUnscaled  
0  2.4872528292500934e-05              2  
1  2.4872528292500934e-05              2  
2                     0.0              1  
3      0.0011192637731625             46  
4                     0.0              1  


In [9]:
g = graphistry.bind()

g = g.bind(
    source='sourceAppId',
    destination='targetAppId',
).edges(merged_edges)

g = g.bind(node='appId', point_label='appId').nodes(merged_nodes)
pallete = [
    'red', 'green', 'blue', 'orange',
    # "#32964d", "#a3d2a0", "#155126", 
    # "#64d4fd", "#378dae", "#374475", 
]
g = g.encode_point_color(
    'type',
    # palette=pallete,
    categorical_mapping={
        "['Palmprint', 'SOTU']": 'blue',
        "['Palmprint']": 'purple',
        "['Taxon', 'Host']": 'green',
    },
    as_categorical=True
)

In [10]:

g.plot()